In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
import random

import h5py
from IPython.utils import io

from joblib import Parallel, delayed
from tqdm import tqdm
import sys

In [2]:
%matplotlib widget

%reload_ext autoreload
%autoreload 2

In [3]:
sys.path.append('../../../../nadavp/new_trap/Tools/')
sys.path.append('../../../../nadavp/new_trap/Tools/StatFramework/')
sys.path.append('../../../../nadavp/opt_lev_analysis/lib/')
import BeadDataFile
from discharge_tools import *

In [4]:
from likelihood_calculator import likelihood_analyser
from likelihood_calculator import GravityFramework
from likelihood_calculator import auxiliary_functions as aux
gfw = GravityFramework.GravityFramework()

In [5]:
import bead_util as bu
# import calib_util as cal
import transfer_func_util as tf
import configuration as config

### Discharge data

In [6]:
ll = likelihood_analyser.LikelihoodAnalyser()
bandwidth = 2
decimate = 10
drive_freq = 41
fsamp = 5000
def func1_scale(i):
    bb = step_file_objs[i]
        
    freq2 = np.fft.rfftfreq(len(bb.other_data[6]), d=1./5000)
    fft_angles = np.angle(np.fft.rfft(bb.other_data[6]))
    phi_tmp2 = (fft_angles[freq2==drive_freq]+np.pi/2)%(2*np.pi)-np.pi
    print(phi_tmp2,np.std(bb.other_data[6])*np.sqrt(2)*50)

    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phi_tmp2, 
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1,
                  'limit_phi': [-2 * np.pi, 2 * np.pi], 
                  'limit_A': [0, 100000], 
                  'print_level': 0, 'fix_f': True, 'fix_phi': True}

    frequency = fit_kwargs['f']

    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
    bb.load_other_data()
    xx2 = signal.filtfilt(b, a, bb.pos_data[1])[::decimate]
    
    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
    return m1_tmp.values[0]

In [7]:
step_cal_dir = r'/data/old_trap/20210603/bead1/discharge/fine_1/'
max_file = 500

In [8]:
step_cal_files = []
for root, dirnames, filenames in os.walk(step_cal_dir):
    for filename in fnmatch.filter(filenames, '*' + config.extensions['data']):
        if '_fpga.h5' in filename:
            continue
        step_cal_files.append(os.path.join(root, filename))
step_cal_files.sort(key = bu.find_str)
num_calib_files = len(step_cal_files)
print(num_calib_files)

# Do the step calibration
step_file_objs = []
for filname in step_cal_files[:-1]:
    try:
        df = bu.DataFile();
        df.load(filname);
        df.load_other_data()
        step_file_objs.append(df);
    except:
        print('***')

142


In [9]:
amps3 = []
for i in range(num_calib_files-1):
    print(i,'/',len(step_cal_files))
    amps3.append(func1_scale(i))

0 / 142
[ 1.46907642] 6.18310351953
1 / 142
[-1.08260669] 6.18387180454
2 / 142
[ 0.52248505] 6.18379312618
3 / 142
[-2.65506352] 6.1839834092
4 / 142
[-3.01028204] 6.184438045
5 / 142
[-0.08399147] 6.18390674379
6 / 142
[-2.60374559] 6.1842820156
7 / 142
[ 1.67527033] 6.18372182051
8 / 142
[ 2.26003184] 6.18439919339
9 / 142
[-2.70789967] 6.1845197363
10 / 142
[-2.86520288] 6.18456439037
11 / 142
[-2.98517603] 6.18521354382
12 / 142
[-2.55315195] 6.18412996861
13 / 142
[-2.36183867] 6.18432416234
14 / 142
[ 1.32091974] 6.18383867204
15 / 142
[-1.14903791] 6.18444724105
16 / 142
[ 2.06185091] 6.18401436622
17 / 142
[ 2.32184315] 6.183666994
18 / 142
[-1.22012927] 6.18453745243
19 / 142
[ 0.76361787] 6.18475149306
20 / 142
[-2.38677996] 6.18461062953
21 / 142
[ 0.49753895] 6.18423914786
22 / 142
[ 2.57170934] 6.18400722769
23 / 142
[-0.7648323] 6.18428722798
24 / 142
[ 1.08825514] 6.18368081019
25 / 142
[ 1.31710337] 6.18428127498
26 / 142
[-1.50431083] 6.18372789618
27 / 142
[ 1.592343

In [10]:
# amps = np.array(amps)
amps3 = np.array(amps3)
_,ax = plt.subplots(figsize=(9.5,4))
scale = 326.26

# Perrinf
ax.scatter(range(len(amps3)), amps3/scale, label='fit phase')
# ax.scatter(range(len(amps3)), amps3/scale, label='fit fixed-phases')

ax.set(xlabel='dataset', ylabel='Charges', ylim=(-0.3,7), xlim=(78,None))
ax.grid(axis='y')
ax.minorticks_on()
ax.grid(which='major', linestyle='-', linewidth='0.5', color='red')
ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
# ax.legend()
from matplotlib.ticker import MaxNLocator
ax.yaxis.set_major_locator(MaxNLocator(integer=True))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
scale = np.abs(np.mean(amps3[87:103]))/4
print("Scale is ", scale)
bb = step_file_objs[-1]
voltage = np.std(bb.other_data[6])*np.sqrt(2)*100
print("voltage: ", voltage)

Scale is  326.524624863
voltage:  12.3688711189


In [338]:
ll = likelihood_analyser.LikelihoodAnalyser()
bandwidth = 2
decimate = 10
drive_freq = 41
def get_amp(i):
    
    fname = r'/data/old_trap/20210603/bead1/neutrality3/w_height_fb_100V_100V_sweep_y_m75V__75V_spinning_xz_100kHz_150V/turbombar_powfb_xyzcool_'+str(i)+'.h5'

    bb = bu.DataFile();
    bb.load(fname);
    bb.load_other_data()
    
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': 0, 
                  'error_A': 1, 'error_f': 10, 'error_phi': 0.5, 'errordef': 1,
                  'limit_phi': [-2 * np.pi, 2 * np.pi], 
                  'limit_A': [0, 10000], 'limit_f': [0, 100], 
                  'print_level': 0, 'fix_f': True, 'fix_phi': False}

    frequency = fit_kwargs['f']

    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
    xx2 = signal.filtfilt(b, a, bb.other_data[6])[::decimate]
    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
#     freq2 = np.fft.rfftfreq(len(bb.other_data[6]), d=1./5000)
#     fft_angles = np.angle(np.fft.rfft(bb.other_data[6]))

    phi_tmp = m1_tmp.values[2]
#     phi_tmp2 = (fft_angles[freq2==drive_freq]+np.pi/2)%(2*np.pi)
#     print(phi_tmp, phi_tmp2, m1_tmp.values[0]*50, np.std(xx2)*np.sqrt(2)*50)
    
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phi_tmp, 
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1,
                  'limit_phi': [-2 * np.pi, 2 * np.pi], 
                  'limit_A': [-100000, 100000], 
                  'print_level': 0, 'fix_f': True, 'fix_phi': True}

    xx2 = signal.filtfilt(b, a, bb.pos_data[1])[::decimate]
    
    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
    return m1_tmp.values[0]

In [339]:
i=0
get_amp(i)

-29.24623132486886

In [310]:
i = 0
fname = r'/data/old_trap/20210603/bead1/neutrality3/w_height_fb_100V_100V_sweep_y_m13V_spinning_xz_100kHz_150V_redo/turbombar_powfb_xyzcool_'+str(i)+'.h5'
zbb = bu.DataFile();
bb.load(fname);
bb.load_other_data()
voltage2 = np.std(bb.other_data[6])*np.sqrt(2)*100
# voltage1=100
print("voltage: ", voltage2, np.mean(bb.other_data[6])*100, len(bb.other_data[1]))

voltage:  99.0188180918 -27.6475184253 50000


In [335]:
amps_w_fb_spin_xz_y_100V_100V_sweep3_m37_5_height_0_4 = []
for i in range(70):
    print(i,'/500')
    try:
        amps_w_fb_spin_xz_y_100V_100V_sweep3_m37_5_height_0_4.append(get_amp(i))
    except:
        print("******")
amps_w_fb_spin_xz_y_100V_100V_sweep3_m37_5_height_0_4 = np.array(amps_w_fb_spin_xz_y_100V_100V_sweep3_m37_5_height_0_4)

0 /500
1 /500
2 /500
3 /500
4 /500
5 /500
6 /500
7 /500
8 /500
9 /500
10 /500
11 /500
12 /500
13 /500
14 /500
15 /500
16 /500
17 /500
18 /500
19 /500
20 /500
21 /500
22 /500
23 /500
24 /500
25 /500
26 /500
27 /500
28 /500
29 /500
30 /500
31 /500
32 /500
33 /500
34 /500
35 /500
36 /500
37 /500
38 /500
39 /500
40 /500
41 /500
42 /500
43 /500
44 /500
45 /500
46 /500
47 /500
48 /500
49 /500
50 /500
51 /500
52 /500
53 /500
54 /500
55 /500
56 /500
57 /500
58 /500
59 /500
60 /500
61 /500
62 /500
63 /500
64 /500
65 /500
66 /500
67 /500
68 /500
69 /500


In [337]:
data = amps_w_fb_spin_xz_y_100V_0V_sweep3_0_50_height_0/scale*voltage/voltage1
data2 = amps_w_fb_spin_xz_y_0V_100V_sweep3_0_50_height_0/scale*voltage/voltage1
data3 = amps_w_fb_spin_xz_y_100V_100V_sweep3_m37_5_height_0_4/scale*voltage/voltage1
_, ax= plt.subplots(figsize=(9.5,4))
# ax.scatter(range(len(data)), data)
# ax.scatter(np.arange(len(data2)), data2)
ax.scatter(np.arange(len(data3)), data3)
ax.plot(np.arange(0,100), np.arange(0,100)*0, '--r')
ax.set(xlabel='Dataset', ylabel=r'$\epsilon$')
# ax.legend()
print(np.mean(data3), np.std(data3)/np.sqrt(len(data3)))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

-0.00298449656316 0.000573455112617


In [237]:
data = amps_w_fb_spin_xz_y_0V_100V_sweep_0_25_height_m1_long_1000[:400]/scale*voltage/voltage1
data2 = amps_w_fb_spin_xz_y_100V_0V_sweep_0_25_height_m1_long_1000[:400]/scale*voltage/voltage1
_, ax= plt.subplots(figsize=(9.5,4))
ax.scatter(range(len(data)), data)
ax.scatter(np.arange(len(data2)), data2)
ax.plot(np.arange(0,400), np.arange(0,400)*0, '--r')
ax.set(xlabel='Dataset', ylabel=r'$\epsilon$')
# ax.legend()
print(np.mean(data))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

-0.0188181024051


In [183]:
data = amps_w_fb_spin_xz_y_100V_100V_sweep_m25V_25V_height_0_4_long_500/scale*voltage/voltage1
data2 = amps_w_fb_spin_xz_y_100V_100V_sweep_m25V_25V_height_m1_long_500/scale*voltage/voltage1
# data2 = amps_w_fb_spin_xz_y_100V_100V_sweep_m25V_25V_height_0_4/scale*voltage/voltage1
# data2 = amps_w_fb_spin_xz_y_100V_100V_sweep_m25V_25V_height_0_4_long_2/scale*voltage/voltage1
# data3 = amps_w_fb_spin_xz_y_95V_105V_sweep_m25V_25V_height_0_4_long_500/scale*voltage/voltage1
data3 = amps_w_fb_spin_xz_y_50V_50V_sweep_m25V_25V_height_m1_long_500/scale*voltage/voltage1*2
_, ax= plt.subplots(figsize=(9.5,4))
ax.scatter(range(len(data)), data)
ax.scatter(np.arange(len(data2)), data2)
ax.scatter(np.arange(len(data3)), data3)
ax.plot(np.arange(0,100), np.arange(0,100)*0, '--r')
ax.set(xlabel='Dataset', ylabel=r'$\epsilon$')
# ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[Text(0.5, 0, 'Dataset'), Text(0, 0.5, '$\\epsilon$')]

### Fit the data

In [298]:
x1 = np.arange(100)
x2 = np.arange(1000)
## estimate sensitivity from identical dataets
z,cov = np.polyfit(x1, data, 1, cov=True)
p1 = np.poly1d(z)
z2, cov2 = np.polyfit(x1, data2, 1, cov=True)
p2 = np.poly1d(z2)
# z3, cov3 = np.polyfit(x1, data3, 1, cov=True)
# p3 = np.poly1d(z3)

In [299]:
cov, cov2, cov3

(array([[  1.62594735e-10,  -8.04843939e-09],
        [ -8.04843939e-09,   5.33879813e-07]]),
 array([[  1.86933225e-10,  -9.25319462e-09],
        [ -9.25319462e-09,   6.13795243e-07]]),
 array([[  2.31818913e-12,  -4.62478732e-10],
        [ -4.62478732e-10,   1.23173502e-07]]))

In [300]:
x = -z2[1]; dx = np.sqrt(cov2[1,1])
y = z2[0]; dy = np.sqrt(cov2[0,0])
dz = np.abs(x/y)*np.sqrt((dx/x)**2+(dy/y)**2)
print(dz)
print(-z[1]/z[0], -z2[1]/z2[0], -z3[1]/z3[0])
print(z[0], z2[0], z3[0])
print(p1(-z2[1]/z2[0]), p1(-z2[1]/z2[0]+dz), p1(-z2[1]/z2[0]-dz))

0.99042769671
44.4536117505 15.1590864857 72.9563426674
-0.000617756631611 0.000818234826546 -0.000144139307458
0.0180968872522 0.0174850439744 0.01870873053


In [314]:
-(z2[1]+z[1])/(z2[0]+z[0])*2/100, 1.5*50/2

(-1.5021903964710268, 37.5)

In [170]:
_, ax= plt.subplots(figsize=(9.5,4))
# ax.scatter(np.arange(len(data)), data, label='-50V to 50V bias sweep - 100V, xz-spin, w/ height fb')
# ax.scatter(np.arange(len(data2)), data2, label='-50V to 50V bias sweep - 100V, xz-spin, w/ height fb')
# ax.scatter(np.arange(len(data3)), data3/scale*scale2, label='-50V to 50V bias sweep - 75V-125V, xz-spin, w/ height fb')
ax.plot(np.arange(0,500), np.arange(0,500)*0, '--r')
ax.set(xlabel='Dataset', ylabel=r'$\epsilon$')
ax.plot(x1, p1(x1), 'b--', label='fit1')
ax.plot(x1, p2(x1), 'r--', label='fit2')
ax.plot(x1, p3(x1), 'r--', label='fit3')
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …